# Project: Clerk


In [16]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [17]:
pip install pypdf -q

Note: you may need to restart the kernel to use updated packages.


In [18]:
pip install docx2txt -q

Note: you may need to restart the kernel to use updated packages.


In [19]:
pip install wikipedia -q

Note: you may need to restart the kernel to use updated packages.


### Loading Documents

In [20]:
# loading PDF, DOCX and TXT files as LangChain Documents
def load_document(file):
    import os
    name, extension = os.path.splitext(file)

    if extension == '.pdf':
        from langchain.document_loaders import PyPDFLoader
        print(f'Loading {file}')
        loader = PyPDFLoader(file)
    elif extension == '.docx':
        from langchain.document_loaders import Docx2txtLoader
        print(f'Loading {file}')
        loader = Docx2txtLoader(file)
    elif extension == '.txt':
        from langchain.document_loaders import TextLoader
        loader = TextLoader(file)
    else:
        print('Document format is not supported!')
        return None

    data = loader.load()
    return data
  

In [21]:
# wikipedia
def load_from_wikipedia(query, lang='en', load_max_docs=2):
    from langchain.document_loaders import WikipediaLoader
    loader = WikipediaLoader(query=query, lang=lang, load_max_docs=load_max_docs)
    data = loader.load()
    return data


### Chunking Data

In [22]:
def chunk_data(data, chunk_size=256):
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=0)
    chunks = text_splitter.split_documents(data)
    return chunks
    

### Calculating Cost

In [23]:
def print_embedding_cost(texts):
    import tiktoken
    enc = tiktoken.encoding_for_model('text-embedding-ada-002')
    total_tokens = sum([len(enc.encode(page.page_content)) for page in texts])
    print(f'Total Tokens: {total_tokens}')
    print(f'Embedding Cost in USD: {total_tokens / 1000 * 0.0004:.6f}')

### Embedding and Uploading to a Vector Database (Pinecone)

In [24]:
def insert_or_fetch_embeddings(index_name):
    import pinecone #library
    from langchain.vectorstores import Pinecone #class
    from langchain.embeddings.openai import OpenAIEmbeddings #text to vectors
    
    embeddings = OpenAIEmbeddings(openai_api_key = "" )
    
    pinecone.init(api_key=os.environ.get('PINECONE_API_KEY'), environment=os.environ.get('PINECONE_ENV'))
    
    if index_name in pinecone.list_indexes():
        print(f'Index {index_name} already exists. Loading embeddings ... ', end='')
        vector_store = Pinecone.from_existing_index(index_name, embeddings)
        print('Ok')
    else:
        print(f'Creating index {index_name} and embeddings ...', end='')
        pinecone.create_index(index_name, dimension=1536, metric='cosine')
        vector_store = Pinecone.from_documents(chunks, embeddings, index_name=index_name)
        print('Ok')
        
    return vector_store
    

In [25]:
def delete_pinecone_index(index_name='all'):
    import pinecone
    pinecone.init(api_key=os.environ.get('PINECONE_API_KEY'), environment=os.environ.get('PINECONE_ENV'))
    
    if index_name == 'all':
        indexes = pinecone.list_indexes()
        print('Deleting all indexes ... ')
        for index in indexes:
            pinecone.delete_index(index)
        print('Ok')
    else:
        print(f'Deleting index {index_name} ...', end='')
        pinecone.delete_index(index_name)
        print('Ok')
    

 # Extra Code That I Have Written

## Santral

In [ ]:

def customer_message(new_message_cust):
  reply_customer = input(new_message_cust)
  input = "From Customer to Sales Representative" + input
  santral(input)



def sales_rep_answers(new_message_sales)
  #chat history'yi nerede tutacağız
  result = ask_with_memory_sr(new_nessage_sales)
  #chat history'yi nerede tutacağız
  santral(result[0])

def technical_partner_answers(new_message_tp)
  #chat history'yi nerede tutacağız
  result = ask_with_memory_sr(new_nessage_tp)
  #chat history'yi nerede tutacağız
  santral(result[0])

def santral(input):
  if "From Main Guy to Technical Partner" in input:
    technical_partner_asnwers("There is one message from your sales representative\n"+input)
  elif "From Technical Partner to Main Guy" in input:
    sales_rep_answers("There is one message from your technical partner\n"+input)
  elif "From Main Guy to Customer" in input:
    customer_message(input.replace("From Main Guy to Customer",""))
  elif "From Customer to Main Guy" in input:
     sales_rep_answers(input)
  else:
    raise Exception("Sth went wrong, you gotta solve it now")

    
    


    
    
    
    




# tp ask with memory

In [ ]:
def ask_with_memory_tp(vector_store, question_tp, chat_history_tp=[]):
    from langchain.chains import ConversationalRetrievalChain #chat history
    from langchain.chat_models import ChatOpenAI

    llm = ChatOpenAI(temperature=1) #by default it uses gpt turbo 3.5
    retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 3})

    crc = ConversationalRetrievalChain.from_llm(llm, retriever) #chain object
    result = crc({'question': question_tp, 'chat_history': chat_history_tp})
    chat_history.append((question_tp, result['answer']))

    return result, chat_history
    #Implement API
    #There will be no vector store
    #Retriever stays here for what

# sr ask with memory

In [ ]:
def ask_with_memory_sr(question_sr, chat_history_sr=[]):
    from langchain.chains import ConversationalRetrievalChain #chat history
    from langchain.chat_models import ChatOpenAI

    llm = ChatOpenAI(temperature=1, model_name = "gpt-4",  openai_api_key = "") #by default it uses gpt turbo 3.5

    crc = ConversationalRetrievalChain.from_llm(llm) #chain object
    result = crc({'question': question_sr, 'chat_history': chat_history_sr})
    chat_history.append((question_sr, result'answer']))

    return result, chat_history


### Asking and Getting Answers

In [26]:
def ask_and_get_answer(vector_store, q):
    from langchain.chains import RetrievalQA
    from langchain.chat_models import ChatOpenAI

    llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=1)

    retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 3})

    chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)
    
    answer = chain.run(q)
    return answer
    
    
def ask_with_memory(vector_store, question, chat_history=[]):
    from langchain.chains import ConversationalRetrievalChain #chat history
    from langchain.chat_models import ChatOpenAI
    
    llm = ChatOpenAI(temperature=1) #by default it uses gpt turbo 3.5
    retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 3})
    
    crc = ConversationalRetrievalChain.from_llm(llm, retriever) #chain object
    result = crc({'question': question, 'chat_history': chat_history})
    chat_history.append((question, result['answer']))
    
    return result, chat_history
    

### Running Code

In [27]:
data = load_document('/Users/adnanbugrametli/Desktop/ChatGPT(Novus)/ChatGPT_Data.pdf')
# print(data[1].page_content)
# print(data[10].metadata)

print(f'You have {len(data)} pages in your data')
print(f'There are {len(data[20].page_content)} characters in the page')

Loading /Users/adnanbugrametli/Desktop/ChatGPT(Novus)/ChatGPT_Data.pdf
You have 67 pages in your data
There are 1074 characters in the page


In [28]:
chunks = chunk_data(data)
print(len(chunks))
# print(chunks[10].page_content)

629


In [29]:
print_embedding_cost(chunks)

Total Tokens: 56619
Embedding Cost in USD: 0.022648


In [31]:
index_name = 'askadocument'
vector_store = insert_or_fetch_embeddings(index_name)

Creating index askadocument and embeddings ...Ok


In [32]:
q = 'What is the whole document about?'
answer = ask_and_get_answer(vector_store, q)
print(answer)

The document is about diamond certification and how experts can help you understand what it means for any created diamond when you book an appointment. It also mentions that having diamond certification ensures that you can focus on the proposal itself instead of worrying.


In [51]:
import time
i = 1
print('Write Quit or Exit to quit.')
while True:
    q = input(f'Question #{i}: ')
    i = i + 1
    if q.lower() in ['quit', 'exit']:
        print('Quitting ... bye bye!')
        time.sleep(2)
        break
    
    answer = ask_and_get_answer(vector_store, q)
    print(f'\nAnswer: {answer}')
    print(f'\n {"-" * 50} \n')

    

Write Quit or Exit to quit.
Question #1: quit
Quitting ... bye bye!


In [43]:
delete_pinecone_index()

Deleting all indexes ... 
Ok


### Ask with Memory Loop

In [ ]:
import time
i = 1

chat_history = []

print("Write Quit or Exit to quit")
while True:
    q = input(f"Question #{i}")
    i = i + 1
    if q.lower() in ["quit","exit"]:
        print("Qutting")
        time.sleep(2)
        break
    result, _ = ask_with_memory(vector_store, q, chat_history)
    print (result['answer'])
    print("----------------------------------------------------------------------")